In [1]:
import pandas as pd
import numpy as np
import re
import requests
import time
import validators
from requests import get
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from time import sleep 
import datetime
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from urllib.parse import urlsplit

In [2]:
######### Google Maps inputs #########

keywords = ['boutique','vetements','mode','shopping','meuble','maquillage','parfum']
rating = "data=!4m2!2m1!6e6" #'!4m4!2m3!5m1!4e9!6e6' # 4 stars and above

zoom = 18 # recommended: 18
base_latitude = 48.852 ### Paris center
base_longitude = 2.346 ### Paris center
city = 'paris' # just for file name

step = 0.01 # 0.005 is roughly 750m
step_number = 3 # how many steps away from base lat x long 

######### Website scraping inputs #########

urls_to_try = ["/contact","/contact-us","/nous-contacter"]


In [3]:
file_name = datetime.now().strftime("%Y-%m-%d") + "_" + city + "_shops.csv"
file_name

'2022-07-18_paris_shops.csv'

In [104]:
def get_them_urls(keywords,rating,base_latitude,base_longitude,step,step_number):

    df_list = []
    
    for h in range(len(keywords)):
        for i in range(-step_number,step_number + 1):
            for j in range(-step_number,step_number + 1):

                lat = base_latitude + step * i
                lon = base_longitude + step * j

                url = "https://www.google.com/maps/search/"+str(keywords[h])+"/@"+str(lat)+','+str(lon)+","+str(zoom)+"z/"+ rating
                print(url)

                ser = Service("C:\Program Files\Google\Chrome\Application\chromedriver.exe")
                op = webdriver.ChromeOptions()
                driver = webdriver.Chrome(service=ser, options=op)
                driver.get(url)

                page_content = driver.page_source
                soup = BeautifulSoup(page_content, "html.parser")

                a_tags = soup.findAll('a', attrs={"aria-label": True})

                shops = []

                for s in range(1,len(a_tags)):
                    shops.append(re.findall('"([^"]*)"', str(a_tags[s]))[0])

                urls = []

                for k in range(len(shops)):

                    shop_name = shops[k] + "\\"

                    ### look for shop name in html
                    ### somewhere before 1st occurence of this name lies the website
                    ### if there is a website populated, it will be in the 2nd [] before shop name

                    string_1 = str(soup)[str(soup).find(shop_name)-500:str(soup).find(shop_name)]

                    idx_start = string_1.rfind("[")
                    string_2 = string_1[0:idx_start]

                    idx_start_2 = string_2.rfind("[")
                    idx_end_2 = string_2.rfind("]")

                    string_3 = string_2[idx_start_2:idx_end_2]

                    shop_url = string_3[3:string_3.find(',')-2]
                    urls.append(shop_url)

                df_shop_data = pd.DataFrame({'name': shops,'url': urls})

                df_list.append(df_shop_data)

    df_shop = pd.concat(df_list)
    return df_shop

def is_url(url):
    return True if validators.url(url) else False

def get_them_emails(url):

    response = None
    contact_email = None
    parts = []
    df_list = []
    
    try:
        response = requests.get(url, timeout = 10)
    except:
        pass
    else:
        parts = urlsplit(url)

    if response != None:
        try:
            response = requests.get(url)
            print(url)
        except:
            pass
        try:
            contact_email = re.findall(r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)',response.text)
        except:
            pass
        try:
            df_list.append(contact_email)
        except:
            pass
        
    if (response != None) & (contact_email == []):
        for i in range(len(urls_to_try)):
            try:
                response = requests.get("{0.scheme}://{0.netloc}".format(parts)+ urls_to_try[i])
                print("{0.scheme}://{0.netloc}".format(parts) + urls_to_try[i])
            except:
                pass
            try:
                contact_email = re.findall(r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)',response.text)
            except:
                pass
            try:
                df_list.append(contact_email)
            except:
                pass
    print (df_list)
    return df_list

def drop_duplicates(lst):
    res = []
    for i in lst:
        if i not in res:
            res.append(i)
    return res

def flatten(xss):
    return [x for xs in xss for x in xs]

def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

blacklist = ['png','jpg', 'example', 'exemple', 'logo','Logo','LOGO','votre']

def clean_list(lst):
    i = 0
    while i < len(lst):
        if (blacklist[0] in lst[i])\
        or (blacklist[1] in lst[i])\
        or (blacklist[2] in lst[i])\
        or (blacklist[3] in lst[i])\
        or (blacklist[4] in lst[i])\
        or (blacklist[5] in lst[i])\
        or (blacklist[6] in lst[i])\
        or (blacklist[7] in lst[i])\
        or has_numbers(lst[i]):
            lst.remove(lst[i])
            i -= 1
        i += 1
    return lst

NameError: name 'soup' is not defined

In [27]:
# df_shop = get_them_urls(keywords,rating,base_latitude,base_longitude,step,step_number)
# df_shop.to_csv('intermediary'+file_name,index=False)

df_shop = pd.read_csv('2022-07-15_paris_shops_intermediary.csv')
df_shop['url'] = df_shop['url'].astype(str)
df_shop.head()

,name,url
0,Dany Boutique,195 Boulevard Brune
1,Boutique de la Sorbonne et des universités de ...,https://www.sorbonne.fr/la-sorbonne/boutique/
2,SELESTIA Boutique - robe de mariée,https://www.selestia-boutique.fr/
3,La Boutique Des Frissons,56 Av. Jean Moulin
4,PRIAM | Boutique de Mariage Paris,http://www.priamboutiques.com/


In [28]:
df_shop['is_url'] = df_shop['url'].apply(is_url)

exclude_list = ["Se connecter","Confidentialité","Conditions","Règles de confidentialité","Conditions d'utilisation"]
exclude_pattern = '|'.join(exclude_list)

df_shop_clean = df_shop[(df_shop.name.str.contains(exclude_pattern) == False) 
                        & (df_shop.url.str.contains("search.google") == False)]

df_shop_clean.loc[df_shop_clean.is_url == False, 'url' ] = "NA"

df_shop_clean = df_shop_clean.drop(columns=['is_url'])
df_shop_clean = df_shop_clean.drop_duplicates(ignore_index= True)

print (str(df_shop_clean.shape[0]) + " rows left down from " + str(df_shop.shape[0]))
df_shop_clean


603 rows left down from 2051


C:\Users\Usuario\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,name,url
0,Dany Boutique,NA
1,Boutique de la Sorbonne et des universités de ...,https://www.sorbonne.fr/la-sorbonne/boutique/
2,SELESTIA Boutique - robe de mariée,https://www.selestia-boutique.fr/
3,La Boutique Des Frissons,NA
4,PRIAM | Boutique de Mariage Paris,http://www.priamboutiques.com/
...,...,...
598,Maison Des Fragrances,https://www.maison-des-fragrances.com/
599,La Compagnie des Parfums de Paris,http://www.compagniedesparfums.com/
600,Fragonard,https://www.fragonard.com/fr-int/magasins/-75/...
601,SEPHORA PARIS GARE DU NORD,https://www.sephora.fr/information-magasin?sto...


In [7]:
# print(response.status_code,response.reason)
# print(response.text)
# print(response.request.headers)

In [29]:
# df_shop_clean['email'] = df_shop_clean['url'].apply(get_them_emails)
# df_shop_email = df_shop_clean
# df_shop_email.to_csv('22-07-16 shops and email.csv',index=False)
# df_shop_email = pd.read_csv('22-07-16 shops and email.csv')
# df_shop_email['email'] = df_shop_email['email'].astype(str)

[]
https://www.sorbonne.fr/la-sorbonne/boutique/
[['boutique.sorbonne@ac-paris.fr']]
https://www.selestia-boutique.fr/
https://www.selestia-boutique.fr/contact
https://www.selestia-boutique.fr/contact-us
https://www.selestia-boutique.fr/nous-contacter
[[], [], [], []]
[]
http://www.priamboutiques.com/
http://www.priamboutiques.com/contact
http://www.priamboutiques.com/contact-us
http://www.priamboutiques.com/nous-contacter
[[], [], [], []]
https://agence.orange.fr/914-boutique-orange-montrouge?utm_source\\u003dgmb
https://agence.orange.fr/contact
https://agence.orange.fr/contact-us
https://agence.orange.fr/nous-contacter
[[], [], [], []]
https://www.instagram.com/sandravoushabille/?hl\\u003dfr
https://www.instagram.com/contact
https://www.instagram.com/contact-us
https://www.instagram.com/nous-contacter
[[], [], [], []]
https://www.arteum.com/fr/museum-products/270/catacombes-de-paris.html
https://www.arteum.com/contact
https://www.arteum.com/contact-us
https://www.arteum.com/nous-cont

https://www.paris.fr/equipements/paris-rendez-vous-17644
https://www.paris.fr/contact
https://www.paris.fr/contact-us
https://www.paris.fr/nous-contacter
[[], [], [], []]
http://www.pachacuirs.fr/
http://www.pachacuirs.fr/contact
http://www.pachacuirs.fr/contact-us
http://www.pachacuirs.fr/nous-contacter
[[], ['ugaltunsoy@gmail.com', 'ugaltunsoy@gmail.com'], [], []]
https://www.facebook.com/coatzze/timeline
https://www.facebook.com/contact
https://www.facebook.com/contact-us
https://www.facebook.com/nous-contacter
[[], [], [], []]
http://www.wolfordshop.fr/
http://www.wolfordshop.fr/contact
http://www.wolfordshop.fr/contact-us
http://www.wolfordshop.fr/nous-contacter
[[], ['service.france@wolford.biz', 'service.france@wolford.biz'], [], []]
http://www.upperconcept.com/
http://www.upperconcept.com/contact
http://www.upperconcept.com/contact-us
http://www.upperconcept.com/nous-contacter
[[], [], [], ['your@email.com']]
http://frenchtrotters.fr/
[['info@frenchtrotters.fr', 'info@frenchtro

https://lateliertextile.fr/
https://lateliertextile.fr/contact
https://lateliertextile.fr/contact-us
https://lateliertextile.fr/nous-contacter
[[], ['contact@lateliertextile.frBUREAUX', 'contact@lateliertextile.fr', 'contact@lateliertextile.fr', 'devis@lateliertextile.fr', 'devis@lateliertextile.fr', 'antoine@lateliertextile.fr', 'antoine@lateliertextile.fr'], [], []]
http://www.moulinrougestore.com/
[['contact@moulinrougestore.com', 'reservation@moulinrouge.fr', 'reservation@moulinrouge.fr', 'contact@moulinrougestore.com']]
https://www.boutique-rugby.com/boutique
https://www.boutique-rugby.com/contact
https://www.boutique-rugby.com/contact-us
https://www.boutique-rugby.com/nous-contacter
[[], [], [], []]
https://petitemendigote.com/
[['logo-black_98eebf4d-3548-49b3-86bd-cc4e8f86f734_200x@2x.png', 'logo-black_98eebf4d-3548-49b3-86bd-cc4e8f86f734_110x@2x.png', 'logo-white_10c131b1-0c9c-422a-a3e2-1f59e96d4ffb_200x@2x.png', 'logo-white_10c131b1-0c9c-422a-a3e2-1f59e96d4ffb_110x@2x.png']]
h

http://www.brissyma.fr/nous-contacter
[[], [], [], []]
https://www.size-factory.com/magasin-grande-taille-homme-paris13e?utm_campaign\\u003dMag-Paris13\\u0026utm_source\\u003dGMB\\u0026utm_medium\\u003dOrganic
https://www.size-factory.com/contact
https://www.size-factory.com/contact-us
https://www.size-factory.com/nous-contacter
[[], [], [], []]
https://www.badrounyparis.com/
[['core-js-bundle@3.2.1', 'focus-within-polyfill@5.0.9', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com', '831126cb46b74583bf6f72c5061cba9d@sentry-viewer.wixpress.com', '8c4075d5481d476e945486754f783364@sentry.io', '8c4075d5481d476e945486754f783364@sentry.io', '8c4075d5481d476e945486754f783364@sentry.io', '8c4075d5481d476e945486754f783364@sentry.io', 'lodash@4.17.21', 'react@16.13.1', 'react-dom@16.13.1']]
[]
[]
https://www.pimkie.fr/magasin-pimkie-00649
https://www.pimkie.fr/contact
https://www.pimkie.fr/contact-us
https://www.pimkie.fr/nous-contacter
[[], [], [], []]
[]
http://stores.ikks.com/fr-fr/b

http://kamilleboutique.fr/contact-us
http://kamilleboutique.fr/nous-contacter
[[], ['kamille0259@gmail.com', 'kamille0259@gmail.com'], [], []]
http://www.anne-elisabeth.com/fr/
http://www.anne-elisabeth.com/contact
http://www.anne-elisabeth.com/contact-us
http://www.anne-elisabeth.com/nous-contacter
[[], [], [], []]
https://www.brucefield.com/
https://www.brucefield.com/contact
https://www.brucefield.com/contact-us
https://www.brucefield.com/nous-contacter
[[], [], [], []]
https://lhabitfrancais.com/
https://lhabitfrancais.com/contact
https://lhabitfrancais.com/contact-us
https://lhabitfrancais.com/nous-contacter
[[], [], [], []]
https://www.cotondoux.com/
https://www.cotondoux.com/contact
https://www.cotondoux.com/contact-us
https://www.cotondoux.com/nous-contacter
[[], [], [], []]
[]
https://www.pimkie.fr/magasin-pimkie-00439
https://www.pimkie.fr/contact
https://www.pimkie.fr/contact-us
https://www.pimkie.fr/nous-contacter
[[], [], [], []]
http://4sq.com/jJx9jF
[['icon-go_foursquare

...

http://www.joiedevivre.fr/
http://www.joiedevivre.fr/contact
http://www.joiedevivre.fr/contact-us
http://www.joiedevivre.fr/nous-contacter
[[], [], [], []]
https://www.giorgettimeda.com/ww/en/atelier-store/paris
[['info@giorgettiparis.fr', 'info@giorgettiparis.fr', 'info@giorgettiparis.fr']]
http://www.flamant.com/
[['info@esign.eu', 'info@flamant.com', 'info@flamant.com']]
https://www.bolia.com/fr-fr/tout-sur-nous/magasins/trouver-des-magasins/bolia-paris/?utm_source\\u003dGMBlisting_Paris\\u0026utm_medium\\u003dorganic
[['saint-germain@bolia.com', 'saint-germain@bolia.com', 'saint-germain@bolia.com', 'serviceclientele@bolia.com', 'serviceclientele@bolia.com', 'serviceclientele@bolia.com', 'serviceclientele@bolia.com']]
https://www.gautier.fr/fr_FR/s/meubles-gautier-kids-bhv-marais
[['bhv@magasins-gautier.fr', 'bhv@magasins-gautier.fr']]
http://www.sensitiveetfils.com/
[['emmanuelle.sensitiveetfils@gmail.com', 'emmanuelle.sensitiveetfils@gmail.com']]
http://www.kaonline.fr/
http://www

http://www.generalstoreparis.com/nous-contacter
[[], ['info@generalstoreparis.com', 'info@generalstoreparis.com', 'info@generalstoreparis.com', 'info@generalstoreparis.com', 'info@generalstoreparis.com', 'info@generalstoreparis.com', 'info@generalstoreparis.com'], [], []]
https://www.kanndesign.com/
https://www.kanndesign.com/contact
https://www.kanndesign.com/contact-us
https://www.kanndesign.com/nous-contacter
[[], [], [], []]
[]
https://confortmeuble.weebly.com/
https://confortmeuble.weebly.com/contact
https://confortmeuble.weebly.com/contact-us
https://confortmeuble.weebly.com/nous-contacter
[[], [], [], []]
http://www.coincanal.net/
http://www.coincanal.net/contact
http://www.coincanal.net/contact-us
http://www.coincanal.net/nous-contacter
[[], [], [], []]
https://www.lelitgigogne.fr/
https://www.lelitgigogne.fr/contact
https://www.lelitgigogne.fr/contact-us
https://www.lelitgigogne.fr/nous-contacter
[[], [], [], []]
https://www.lelitgigogne.fr/
https://www.lelitgigogne.fr/contact

https://heloiselescure.fr/nous-contacter
[[], ['lescureheloisepro@gmail.com', 'heloise@heloiselescure.fr', 'santiu.pro@gmail.com'], ['santiu.pro@gmail.com'], ['santiu.pro@gmail.com']]
https://www.mademoiselle-bio.com/boutique-institut/saint-antoine
https://www.mademoiselle-bio.com/contact
https://www.mademoiselle-bio.com/contact-us
https://www.mademoiselle-bio.com/nous-contacter
[[], [], [], []]
http://art-liner.fr/
[['art-liner-logo@2x-copie.png', 'art-liner-logo@2x-copie.png', 'art-liner-logo@2x-copie.png', 'art-liner-logo@2x-copie.png', 'art-liner-logo@2x-copie-copie.png', 'art-liner-logo@2x-copie-copie.png', 'art-liner-logo@2x-copie-copie.png', 'art-liner-logo@2x-copie-copie.png', 'art-liner-logo@2x-copie-copie.png', 'art-liner-logo@2x-copie-copie.png', 'artliner-logo-blanc@2x-copie.png', 'artliner-logo-blanc@2x-copie-300x123.png', 'artliner-logo-blanc@2x-copie-200x82.png', 'artliner-logo-blanc@2x-copie.png']]
http://www.makemybeauty.com/
http://www.makemybeauty.com/contact
http://

https://www.essentialparfums.com/
https://www.essentialparfums.com/contact
https://www.essentialparfums.com/contact-us
https://www.essentialparfums.com/nous-contacter
[[], [], [], []]
https://www.dior.com/fr_fr/parfum/nouveautes-parfum
[['search-insights@1.4.0', 'contactdior@dior.com', 'dpo_france@diormail.com', 'contactdior@dior.com', 'contactdior@dior.com', 'christian@dior.com']]
http://www.parlemoideparfum.com/
[['parle-moi-de-parfum-wake-up-world-visu-3-300x238@2x.jpg', 'parle-moi-de-parfum-mile-high-38-a-1-150x150@2x.jpg', 'parle-moi-de-parfum-mile-high-38-a-1-300x300@2x.jpg', 'parle-moi-de-parfum-mile-high-38-a-1-250x250@2x.jpg', 'parle-moi-de-parfum-mile-high-38-1-1-150x150@2x.jpg', 'parle-moi-de-parfum-mile-high-38-1-1-300x300@2x.jpg', 'parle-moi-de-parfum-mile-high-38-1-1-250x250@2x.jpg', 'parle-moi-de-parfum-saffron-wood-91-2-150x150@2x.jpg', 'parle-moi-de-parfum-saffron-wood-91-2-300x300@2x.jpg', 'parle-moi-de-parfum-saffron-wood-91-2-250x250@2x.jpg', 'parle-moi-de-parfum-sa

[]
http://parfumsetsenteursdupaysbasque.com/
http://parfumsetsenteursdupaysbasque.com/contact
http://parfumsetsenteursdupaysbasque.com/contact-us
http://parfumsetsenteursdupaysbasque.com/nous-contacter
[[], [], [], []]
[]
https://www.parfumsurmesure.com/
[['contact@stephaniedebruijn.com']]
https://services.chanel.com/fr_FR/storelocator/store/le+bon+marche-46160.html?y_source\\u003d1_MTQ5MzA2MDAtNzE1LWxvY2F0aW9uLndlYnNpdGU%3D
https://services.chanel.com/contact
https://services.chanel.com/contact-us
https://services.chanel.com/nous-contacter
[[], [], [], []]
[]
[]
http://www.goutalparis.com/
[['vendors@layout.fullspots.min', 'vendors@layout.fullspots.min.css', 'vendors@layout.fullspots.min', 'vendors@layout.fullspots.min.js', 'vendors@layout.theme.min.js']]
http://www.lelabofragrances.com/
http://www.lelabofragrances.com/contact
http://www.lelabofragrances.com/contact-us
http://www.lelabofragrances.com/nous-contacter
[[], [], [], []]
[]
http://www.nicolasdanila.net/
[['bluebird@3.5.3', 

In [106]:
pd.set_option('display.max_rows', 1000)
df_shop_email['email_clean'] = df_shop_email.email
df_shop_email['email_clean'] = df_shop_email['email_clean'].apply(flatten)
df_shop_email['email_clean'] = df_shop_email['email_clean'].apply(clean_list)
df_shop_email['email_clean'] = df_shop_email['email_clean'].apply(drop_duplicates)
# df_shop_email['email_clean'] = df_shop_email['email_clean'].apply(clean)
df_shop_email

,name,url,email,email_clean
0,Dany Boutique,NA,[],[]
1,Boutique de la Sorbonne et des universités de ...,https://www.sorbonne.fr/la-sorbonne/boutique/,[[boutique.sorbonne@ac-paris.fr]],[boutique.sorbonne@ac-paris.fr]
2,SELESTIA Boutique - robe de mariée,https://www.selestia-boutique.fr/,"[[], [], [], []]",[]
3,La Boutique Des Frissons,NA,[],[]
4,PRIAM | Boutique de Mariage Paris,http://www.priamboutiques.com/,"[[], [], [], []]",[]
5,Boutique Orange - Montrouge,https://agence.orange.fr/914-boutique-orange-m...,"[[], [], [], []]",[]
6,Boutique Sandra Vous Habille,https://www.instagram.com/sandravoushabille/?h...,"[[], [], [], []]",[]
7,Shop Paris Catacombs,https://www.arteum.com/fr/museum-products/270/...,"[[], [], [], []]",[]
8,agnès b. Paris Vieux Colombier Store,https://www.agnesb.eu/stores/france/paris/vieu...,"[[], [personal-shopper@agnesb.fr], [], []]",[personal-shopper@agnesb.fr]
9,MS Boutique,http://www.msboutique-decoration-paris.fr/,"[[], [], [], []]",[]


In [107]:
df_shop_email.describe()

,name,url,email,email_clean
count,603,603,603,603
unique,571,474,249,188
top,Sephora,NA,"[[], [], [], []]",[]
freq,6,110,171,395


In [108]:
df_shop_email.to_csv(file_name,index=False)